# Notebook to conduct ML related tasks

# Imports

#### Standard library imports

In [1]:
import sys
sys.path.append("../")
import os
import pickle

#### Third party imports

In [2]:
import boto3
import kaggle
import pandas as pd
import numpy as np

#### Local application imports

In [3]:
%load_ext autoreload
%autoreload 2

from pkg_dir.config import *
from pkg_dir.src.utils import *
from pkg_dir.src.functions import *
from pkg_dir.src.parameters import *

# Segmented pipeline

## Extract

In [ ]:
extract_pipeline_func()

In [ ]:
extract_objects = os.listdir(pipeline_pkl_extract_local_dir)

In [ ]:
bucket_name = base_bucket_name
extract_bucket_key = pipeline_pkl_extract_aws_key
extract_obj = 'extract_train.pkl'

dfx = read_s3_obj_to_variable(bucket_name, extract_bucket_key, extract_obj)

In [ ]:
dfx.set_index('PassengerId', inplace=True)

In [ ]:
dfx.loc[:, 'Transported'].copy()

## Transform

In [6]:
bucket_name = base_bucket_name
extract_bucket_key = pipeline_pkl_transform_aws_key
extract_obj = 'trans_train_x.pkl'

dfx = read_s3_obj_to_variable(bucket_name, extract_bucket_key, extract_obj)

In [ ]:
dfx.info()

## Feature engineering

In [ ]:
pkl_path = pipeline_pkl_transform_local_dir + 'trans_train_x.pkl'

In [ ]:
with open(pkl_path, 'rb') as obj:
    dfx = pickle.load(obj)

In [ ]:
exp_cols = [
    'RoomService',
    'FoodCourt',
    'ShoppingMall',
    'Spa',
    'VRDeck',
]

dfx.loc[:, exp_cols].sum(axis=1)

In [ ]:
dfx.columns.tolist().index('VRDeck')

In [66]:
len(titanicsp_data_schema)

14

In [63]:
lx = [
    col
    for col in titanicsp_data_schema
    if
        'model_relevant' in titanicsp_data_schema[col]
    and
        titanicsp_data_schema[col]['model_relevant']
]

lx

['HomePlanet',
 'CryoSleep',
 'Cabin',
 'Destination',
 'Age',
 'VIP',
 'RoomService',
 'FoodCourt',
 'ShoppingMall',
 'Spa',
 'VRDeck',
 'Name']

In [ ]:
len(lx)

# Header 1

# *Notes*

## Testing interaction with AWS s3

### Setting the s3 client

In [ ]:
creds = read_yaml(creds_file_path)

In [ ]:
dev = boto3.Session(
    aws_access_key_id=creds['aws']['aws_access_key_id'],
    aws_secret_access_key=creds['aws']['aws_secret_access_key'],
)

In [ ]:
s3 = dev.client('s3')

### Listing buckets

In [ ]:
s3.list_buckets()

### Uploading file to bucket

#### Pre-created file

In [ ]:
## Uploading - test 1
file_path = dataset_dir + '/test_file.txt'
bucket = 'titanic-spaceship-aws-bucket'
upload_file(file_path, bucket, object_name=None)

In [ ]:
## Uploading - test 2
file_path = dataset_dir + '/test_file.txt'
bucket = 'titanic-spaceship-aws-bucket'
object_name = 'test_folder' + '/test_file.txt'
upload_file(file_path, bucket, object_name)

### Listing objects in bucket

In [ ]:
s3 = create_s3_client()

In [ ]:
bucket_name = base_bucket_name

In [ ]:
bucket_name = base_bucket_name
bucket_key = 'pipeline_pkls/extract'

[
    obj['Key'].split(sep='/')[-1]
    for obj
    in s3.list_objects_v2(Bucket=bucket_name, Prefix=bucket_key)['Contents']
]

In [ ]:
lx = list_objects_in_bucket_key(bucket_name, bucket_key)
lx

### Reading file from bucket

In [ ]:
s3 = create_s3_client()

In [ ]:
bucket_name = bucket_name
# bucket_key = os.path.join(aws_pipeline_pkl_extract, pipeline_pkl_extract_name)
bucket_key = 'pipeline_pkls/extract/extract_train.pkl'

In [ ]:
obj = s3.get_object(Bucket=bucket_name, Key=bucket_key)

In [ ]:
dfx = pickle.loads(obj['Body'].read())

## Using kaggle library

In [ ]:
kaggle.api.authenticate()

In [ ]:
kaggle.api.competition_download_files

In [ ]:
kaggle.api.competition_download_files(
    'spaceship-titanic', 
    path=dataset_dir, 
)

## Header 2

---

---